In [1]:
import re
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, precision_score
from sklearn.utils import shuffle

In [2]:
feature_type = '1'

#1 -> just sentence
#2 -> all text
#3 -> all text + other columns

In [3]:
target_type = 2

#1 -> all label
#2 -> pos + neg
#3 -> pos + neg(negative+netural)

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')

In [6]:
train.head()

,text,sentence,snp,phenotype,ASSOCIATION,CONFIDENCE
0,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,nicotine metabolism,neutral,zero
1,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,fetal growth characteristics,positive,weak
2,OBJECTIVE: Maternal smoking during pregnancy i...,RESULTS: Among mothers who did not smoke durin...,rs1051730,fetal growth characteristic,negative,-
3,OBJECTIVE: Maternal smoking during pregnancy i...,Among mothers who continued smoking during pre...,rs1051730,head circumference,negative,-
4,OBJECTIVE: Maternal smoking during pregnancy i...,The T-allele of maternal rs1051730 was associa...,rs1051730,second and third trimester fetal femur length,positive,weak


In [7]:
train = train[~train.ASSOCIATION.isin(['neutral'])]
test  = test[~test.ASSOCIATION.isin(['neutral'])]

In [8]:
# dic = {"neutral": "negative"}

# train.replace({"ASSOCIATION": dic}, inplace=True)
# test.replace({"ASSOCIATION": dic}, inplace=True)

In [9]:
train['label'] = train['ASSOCIATION']
test['label']  = test['ASSOCIATION']

In [10]:
train.dropna(subset=['sentence'], inplace=True)
test.dropna(subset=['sentence'], inplace=True)

In [11]:
res = pd.read_excel('res.xlsx')

In [12]:
def vectorize_bow(df_X, cv):
    bow_X = cv.transform(df_X)
    bow_df = pd.DataFrame(data=bow_X.toarray(),
                          columns=[cv.get_feature_names_out()])
    return bow_df

def define_and_run_model(model, X_train, X_test, y_train):
    clf = model
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

def show_result(y_pred, y_test):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))
    print(accuracy_score(y_test, y_pred))
    
def save_res(model_name, feature_type, target_type, y_pred_sr, y_test_sr, idx):
    res_pre = precision_score(y_test_sr, y_pred_sr, average='weighted')
    res_rec = recall_score(y_test_sr, y_pred_sr, average='weighted')
    res_f1 = f1_score(y_test_sr, y_pred_sr, average='weighted')
    res.at[idx, 'model-name'] = model_name
    res.at[idx, 'feature-type'] = feature_type
    res.at[idx, 'target-type'] = target_type
    res.at[idx, 'accuracy'] = accuracy_score(y_test_sr, y_pred_sr)
    res.at[idx, 'precision'] = res_pre
    res.at[idx, 'recall'] = res_rec
    res.at[idx, 'f1-score'] = res_f1

In [13]:
possible_labels = train.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

train['label'] = train.label.replace(label_dict)
test['label']  = test.label.replace(label_dict)

In [14]:
df = pd.concat([train, test])

In [15]:
bow_vectorizer = CountVectorizer(lowercase=False, ngram_range=(1, 1), max_df=.80, min_df=1)
bow_vectorizer.fit(df['sentence'])

CountVectorizer(lowercase=False, max_df=0.8)

In [16]:
train_bow_df  = vectorize_bow(list(train['sentence']), bow_vectorizer)
test_bow_df   = vectorize_bow(list(test['sentence']), bow_vectorizer)

In [17]:
X_train = train_bow_df
y_train = train['label']

X_test = test_bow_df
y_test = test['label']

In [18]:
X_train.shape

(792, 2534)

In [19]:
y_train.shape

(792,)

In [20]:
X_test.shape

(199, 2534)

In [21]:
y_test.shape

(199,)

In [22]:
train['ASSOCIATION'].value_counts()

positive    701
negative     91
Name: ASSOCIATION, dtype: int64

In [23]:
test['ASSOCIATION'].value_counts()

positive    170
negative     29
Name: ASSOCIATION, dtype: int64

In [24]:
%%time
smodel = SVC(kernel='rbf')
smodel.fit(X_train, y_train)
pred_values_svm = smodel.predict(X_test)
show_result(pred_values_svm, y_test)
save_res('svm', feature_type, target_type, pred_values_svm, y_test, len(res))

[[170   0]
 [ 27   2]]
              precision    recall  f1-score   support

           0      0.863     1.000     0.926       170
           1      1.000     0.069     0.129        29

    accuracy                          0.864       199
   macro avg      0.931     0.534     0.528       199
weighted avg      0.883     0.864     0.810       199

0.864321608040201
Wall time: 542 ms


In [25]:
%%time
lr = LogisticRegression(random_state=0,
                        penalty='l2',
                        solver='newton-cg',
                        C=1.2,
                        max_iter=20)
lr.fit(X_train, y_train)
pred_values_lr = lr.predict(X_test)
show_result(pred_values_lr, y_test)
save_res('lr', feature_type, target_type, pred_values_lr, y_test, len(res))

[[170   0]
 [ 24   5]]
              precision    recall  f1-score   support

           0      0.876     1.000     0.934       170
           1      1.000     0.172     0.294        29

    accuracy                          0.879       199
   macro avg      0.938     0.586     0.614       199
weighted avg      0.894     0.879     0.841       199

0.8793969849246231
Wall time: 441 ms


In [26]:
%%time
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
pred_values_rf = model.predict(X_test)
show_result(pred_values_rf, y_test)
save_res('rf', feature_type, target_type, pred_values_rf, y_test, len(res))

[[170   0]
 [ 28   1]]
              precision    recall  f1-score   support

           0      0.859     1.000     0.924       170
           1      1.000     0.034     0.067        29

    accuracy                          0.859       199
   macro avg      0.929     0.517     0.495       199
weighted avg      0.879     0.859     0.799       199

0.8592964824120602
Wall time: 906 ms


In [27]:
import os
os.remove("res.xlsx") 

res.to_excel('res.xlsx', index=False)